In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pandas import DataFrame, read_csv
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sb

In [ ]:
path = r'../input/house-prices-advanced-regression-techniques/train.csv'


In [ ]:
df = pd.read_csv(path)
df.head()

In [ ]:
# df.shape
df =df.sort_index(axis=1)
(df.info())


In [ ]:
a=(df.isnull().sum())
# a=(a.to_frame())
# a=a.rename(columns={0:'col'})
# a = a[a.col!=0]
# a.shape
b=a[a!=0]
b

In [ ]:
# df['MSZoning'].value_counts()
df['MiscFeature'].value_counts()

In [ ]:
sb.heatmap(df.isnull(), yticklabels=False, cbar=False,cmap='viridis')

In [ ]:
def filling(dataf, colS, operation):
    if operation.lower()=='mode':
        for col in colS:
            dataf[col] = dataf[col].fillna(dataf[col].mode()[0])
    elif operation.lower() == 'mean':
        for col in colS:
            dataf[col] = dataf[col].fillna(dataf[col].mean())
    return dataf

In [ ]:
df = filling(df,['LotFrontage','GarageYrBlt','MasVnrArea'], 'mean')


In [ ]:
df = filling(df, ['BsmtCond','BsmtExposure', 'BsmtFinType1','BsmtFinType2',
                  'BsmtQual','Electrical','FireplaceQu','GarageCond',
                 'GarageFinish','GarageQual','GarageType','MasVnrType'], 'mode')


In [ ]:
df.drop(['Alley','Fence','MiscFeature','PoolQC','GarageYrBlt','Id'],axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
a=df.isnull().sum()
b=a[a!=0]
b
a

In [ ]:
sb.heatmap(df.isnull(),yticklabels=False,cbar=False, cmap='YlGnBu')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
main_df = df.copy()

In [ ]:
dfs = df.loc[:,df.dtypes==object]
cat_cols = list(dfs.columns)
dfs.head()

In [ ]:
dfss = df.loc[:,df.dtypes !=object]
int_col = list(dfss.columns)
dfss.head()

In [ ]:
test_path = r'../input/house-prices-advanced-regression-techniques/test.csv'
testdf = pd.read_csv(test_path)
testdf = testdf.sort_index(axis=1)
testdf.head()

In [ ]:
testdf.shape

In [ ]:
testdf = filling(testdf,['LotFrontage','GarageYrBlt','MasVnrArea'], 'mean')
testdf = filling(testdf, ['BsmtCond','BsmtExposure', 'BsmtFinType1','BsmtFinType2',
                  'BsmtQual','Electrical','FireplaceQu','GarageCond',
                 'GarageFinish','GarageQual','GarageType','MasVnrType'], 'mode')
testdf.drop(['Alley','Fence','MiscFeature','PoolQC','GarageYrBlt','Id'],axis=1,inplace=True)

In [ ]:
testdf.shape

In [ ]:
testdf.to_csv('cleaned_test.csv',index=False)

In [ ]:
final_df = pd.concat([main_df,testdf],axis=0)

In [ ]:
final_df['SalePrice']

In [ ]:
final_df.shape

In [ ]:
def category_onehot_multcols(dframe,multcolumns):
    df_final=dframe
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(final_df[fields],drop_first=True)
        
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([final_df,df_final],axis=1)
        
    return df_final

In [ ]:
final_df = category_onehot_multcols(final_df,cat_cols)

In [ ]:
final_df.head()
# final_df.columns

In [ ]:
final_df = final_df.loc[:,~final_df.columns.duplicated()]
final_df.shape

In [ ]:
traindf = final_df.iloc[:1460,:]
testdf = final_df.iloc[1460:,:]

In [ ]:
traindf.shape
testdf.shape

In [ ]:
xtrain= traindf.drop(['SalePrice'],axis=1)
ytrain=traindf['SalePrice']


In [ ]:
xtest = testdf.drop(['SalePrice'],axis=1)


In [ ]:
import xgboost

classifier = xgboost.XGBRegressor()
classifier.fit(xtrain,ytrain)

In [ ]:
ypred= classifier.predict(xtest)
ypred

In [ ]:
submis = r'../input/house-prices-advanced-regression-techniques/sample_submission.csv'

submit = read_csv(submis)
yPred = DataFrame(ypred)
finaldf = pd.concat([submit['Id'],yPred],axis=1)
finaldf.columns=['Id','SalePrice']
finaldf.to_csv('predict_price.csv',index=False)

In [ ]:
finaldf